In [14]:
import random

print(" ".join([str(random.randint(1,1)) for i in range(3000)]))

In [ ]:
arr = [1]*3000
arr[:1500:2] = range(2,750+2)
arr[1500::2] = range(2,750+2)
print(" ".join(str(x) for x in arr))